<div class='alert alert-info' style='text-align: center'><h1>Cropping Chest X-Rays</h1>
    - yet another chest x-ray processing notebook -
</div>

### Simple CXR cropping technique

- The idea here is to easily remove noise around the edges of a chest x-ray without using segmentation.
- Since we know the edges of a CXR are generally very dark compared to the areas with tissue in them, we can remove pixel rows that are below a certain threshold and crop the image accordingly.
- This is a rough idea and would require some tuning to work in production, but it could produce some very usable results.

#### Let's grab an image, resize it and scale it to 8 bits (for simplicity), then attempt to crop down to the lung fields.

In [ ]:
import numpy as np
import pydicom
import cv2
import matplotlib.pyplot as plt
from skimage.transform import resize

In [ ]:
# Grab a random DICOM file from the SIIM Covid-19 Detection set
img_file = "../input/siim-covid19-detection/train/00792b5c8852/1f52bcb3143e/3fadf4b48db3.dcm"
img = pydicom.dcmread(img_file)

In [ ]:
# Resize the pixels
w = int(img.pixel_array.shape[0] * .25)
h = int(img.pixel_array.shape[1] * .25)

px = img.pixel_array / 255
img = resize(px, (w, h), anti_aliasing=True).astype(float)

# scale the pixels
img = (np.maximum(img,0) / img.max()) * 255.0
img = np.uint8(img)

# TODO: Invert MONOCHROME1 images here.

In [ ]:
# Display the original image
plt.figure(figsize=(15,5))
plt.imshow(img,cmap="gray");

In [ ]:
# Make a binarized copy of the image
thresh = 150
img_bin = cv2.threshold(img, thresh, 255, cv2.THRESH_BINARY)[1]

plt.figure(figsize=(15,5))
plt.imshow(img_bin,cmap="gray");

In [ ]:
# Flip the image 90 degrees
img_bin = cv2.rotate(img_bin, cv2.cv2.ROTATE_90_CLOCKWISE)
plt.figure(figsize=(15,5))
plt.imshow(img_bin,cmap="gray");

- With the image flipped, we can extract rows and calculate their mean. 
- Any row that is 'too dark' is probably not part of the patient and can be cropped out.

In [ ]:
right = 0;
left = 0;
line_thickness = 2

# This is the value that specifies how bright a row is to consider it 'not the edge (too bright)'
intensity_threshold = 190

# Start at the bottom and work upwards checking the mean of pixels in every 10th row, this is the right side of the image
for i in range(img_bin.shape[0]-1,0,-10):
    row_mean = img_bin[i].mean()
    if row_mean > intensity_threshold:
        right = i
        
        # Draw a line where we want to crop
        cv2.line(img_bin, (0, i), (img_bin.shape[1], i), (0, img_bin.shape[1], 0), thickness=line_thickness)
        break
        
# Start at the top and go down to find the left side
for i in range(0,img_bin.shape[0]-1,10):
    row_mean = img_bin[i].mean()
    if row_mean > intensity_threshold:
        left = i
        
        # Draw a line where we want to crop
        cv2.line(img_bin, (0, i), (img_bin.shape[1], i), (0, img_bin.shape[1], 0), thickness=line_thickness)
        break

In [ ]:
# Draw lines on the image where the mean intensity is > intensity_threshold
plt.figure(figsize=(15,5))
plt.imshow(img_bin,cmap="gray");

- The two lines on the image represent where the row became much 'brighter'. This is where we'll crop.

In [ ]:
# Rotate the image back to it's normal orientation
img_bin = cv2.rotate(img_bin, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)

x1 = left
y1 = 0
x2 = right
y2 = img_bin.shape[1]

# Grab the region we identified from the binarized image
img_cropped = img_bin[y1:y2, x1:x2]
plt.figure(figsize=(15,5))
plt.imshow(img_cropped,cmap="gray");

- Now we'll do the same process to find the best place to crop the top and bottom.

In [ ]:
top = 0;
bottom = 0;

# Set some threshold values to specify what we consider edge vs patient
bright_threshold = 240
dark_threshold = 100

# Start at the bottom and work upward
for i in range(img_cropped.shape[0]-1,0,-10):
    row_mean = img_cropped[i].mean()
    if row_mean < bright_threshold:
        # Add 100 pixels of padding so we don't cut the costophrenic angles off
        bottom = i + 100
        cv2.line(img_cropped, (0, bottom), (img_cropped.shape[1], bottom), (0, img_cropped.shape[1], 0), thickness=line_thickness)
        break
        
# Start at the top and go down
for i in range(0,img_cropped.shape[0]-1,10):
    row_mean = img_cropped[i].mean()
    if row_mean > dark_threshold:
        top = i
        cv2.line(img_cropped, (0, i), (img_cropped.shape[1], i), (0, img_cropped.shape[1], 0), thickness=line_thickness)
        break
        
plt.figure(figsize=(15,5))
plt.imshow(img_cropped,cmap="gray");

In [ ]:
x1 = 0
y1 = top
x2 = img_bin.shape[0]
y2 = bottom

img_cropped = img_cropped[y1:y2, x1:x2]
plt.figure(figsize=(15,5))
plt.imshow(img_cropped,cmap="gray");

In [ ]:
# Display the original image and the cropped section
img_cropped = img[top:bottom, left:right]
plt.figure(figsize=(15,5))

plt.subplot(1, 2, 1)
plt.imshow(img,cmap="gray");

plt.subplot(1, 2, 2)
plt.imshow(img_cropped,cmap="gray");

- This isn't a perfect method, but hopefully someone can expand on it.

**Here are some other processing notebooks I made:**
- Lung Segmentation Without CNN -> https://www.kaggle.com/davidbroberts/lung-segmentation-without-cnn
- Applying filters to x-rays -> https://www.kaggle.com/davidbroberts/applying-filters-to-chest-x-rays
- Rib supression on Chest X-Rays -> https://www.kaggle.com/davidbroberts/rib-suppression-poc
- Manual DICOM VOI LUT -> https://www.kaggle.com/davidbroberts/manual-dicom-voi-lut
- Apply Unsharp Mask to Chest X-Rays -> https://www.kaggle.com/davidbroberts/unsharp-masking-chest-x-rays
- Bounding Boxes on Cropped Images -> https://www.kaggle.com/davidbroberts/bounding-boxes-on-cropped-images
- Visualizing Chest X-Ray bit planes -> https://www.kaggle.com/davidbroberts/visualizing-chest-x-ray-bitplanes
- DICOM full range pixels as CNN input -> https://www.kaggle.com/davidbroberts/dicom-full-range-pixels-as-cnn-input
- Standardizing Chest X-Ray Dataset Exports -> https://www.kaggle.com/davidbroberts/standardizing-cxr-datasets